In [3]:
import pandas as pd
import numpy as np

In [4]:
import os
dataset = np.array([])
def data_collect(name):
    df = pd.read_pickle(os.path.abspath(name))
    df_labels = df['label']
    df_labels = np.reshape(df_labels,(len(df['label']),1))
    data = np.concatenate((df['signal']['chest']['ACC'],df['signal']['chest']['ECG'],df['signal']['chest']['EMG'],df['signal']['chest']['EDA'],df['signal']['chest']['Temp'],df['signal']['chest']['Resp'],df_labels),axis = 1) 
    data = np.array([data[i] for i in range(len(data[:,8])) if 4 >= data[i,8] > 0 ])
    return data
for i in range(2,3):
    name = 'C:\\Users\\DELL\\Desktop\\PR\\WESAD\\S'+str(i) + '\\S'+str(i)+'.pkl'
    if i == 2 : 
        dataset = data_collect(name)
        print(i)
    else :
        if i != 12 :
            dataset = np.concatenate((dataset,data_collect(name)),axis = 0)
            print(i)

2


In [5]:
df = pd.DataFrame({'ACC_X':dataset[:,0],'ACC_Y':dataset[:,1],'ACC_Z':dataset[:,2],'ECG':dataset[:,3],'EDA':dataset[:,4],'EMG':dataset[:,5],'RESP':dataset[:,6],'TEMP':dataset[:,7],'TARGET':dataset[:,8]})

In [6]:
df

,ACC_X,ACC_Y,ACC_Z,ECG,EDA,EMG,RESP,TEMP,TARGET
0,0.8914,-0.1102,-0.2576,0.030945,-0.003708,5.710983,29.083618,1.191711,1.0
1,0.8926,-0.1086,-0.2544,0.033646,-0.014145,5.719376,29.122437,1.139832,1.0
2,0.8930,-0.1094,-0.2580,0.033005,0.010208,5.706406,29.115234,1.141357,1.0
3,0.8934,-0.1082,-0.2538,0.031815,0.012634,5.712509,29.126709,1.155090,1.0
4,0.8930,-0.1096,-0.2570,0.030350,0.002060,5.727005,29.100861,1.133728,1.0
...,...,...,...,...,...,...,...,...,...
2022294,0.4378,-0.2348,-0.8380,-0.182602,-0.015793,0.484085,31.926239,-1.609802,4.0
2022295,0.4378,-0.2338,-0.8394,-0.170609,0.000687,0.473404,31.932190,-1.646423,4.0
2022296,0.4388,-0.2338,-0.8386,-0.160812,0.004532,0.463486,31.918823,-1.643372,4.0
2022297,0.4398,-0.2374,-0.8390,-0.156326,0.000595,0.459290,31.932190,-1.661682,4.0


In [7]:
df_new = pd.DataFrame()

In [8]:
from scipy.signal import find_peaks,peak_widths

In [9]:
import numpy.fft as fft

In [10]:
for i in range(int(len(df)/700)):
    x = df.iloc[i:i+int(len(df)/700),:]
    mn_acc_x = x['ACC_X'].mean()
    #rint(mn_acc_x)
    mn_acc_y = x['ACC_Y'].mean()
    #rint(mn_acc_y)
    mn_acc_z = x['ACC_Z'].mean()
    #rint(mn_acc_z)
    std_acc_x = x['ACC_X'].std()
    #rint(std_acc_x)
    std_acc_y = x['ACC_Y'].std()
    # int(std_acc_y)
    std_acc_z = x['ACC_Z'].std()
    #td_acc_z)
    itgl_acc_x = np.trapz(x['ACC_X'])
    #rint(itgl_acc_x)
    itgl_acc_y = np.trapz(x['ACC_Y'])
    #int(itgl_acc_y)
    itgl_acc_z = np.trapz(x['ACC_Z'])
    #(itgl_acc_z)
    mn_emg = x['EMG'].mean()
    #rint(mn_emg)
    std_emg = x['EMG'].std()
    #rint(std_emg)
    max_emg = np.amax(x['ACC_X'])
    min_emg = np.amin(x['ACC_X']) 
    dr_emg = max_emg/min_emg
    #rint(dr_emg)
    itgl_emg = np.trapz(x['EMG'])
    med_emg = np.median(x['EMG'])
    per_10_emg = np.percentile(x['EMG'],10)
    per_90_emg = np.percentile(x['EMG'],90)
    peaks,properties = find_peaks(x['EMG'],height = 0)
    mn_pk_amp = np.mean(properties['peak_heights'])
    std_pk_amp = np.std(properties['peak_heights'])
    sum_pk_amp = np.sum(properties['peak_heights'])
    mn_temp = x['TEMP'].mean()
    std_temp = x['TEMP'].std()
    max_temp = x['TEMP'].max()
    min_temp = x['TEMP'].min()
    dr_temp = max_temp/min_temp
    mn_eda = x['EDA'].mean()
    std_eda = x['EDA'].std()
    max_eda = x['EDA'].max()
    min_eda = x['EDA'].min()
    dr_eda = max_eda/min_eda
    peaks,properties = find_peaks(x['EDA'],height = 0)
    mn_scr = np.mean(properties['peak_heights'])
    std_scr = np.std(properties['peak_heights'])   
    num_scr = np.size(peaks)
    sum_scr = np.sum(properties['peak_heights'])
    width_scr = peak_widths(x['EDA'], peaks, rel_height=0)
    #pltu_scr = np.array(properties['plateau_sizes'])
    ht_scr = properties['peak_heights']
    ar_scr = 0.5*np.matmul(ht_scr,width_scr[1])
    df_new = df_new.append({'mean_accx':mn_acc_x,'mean_accy':mn_acc_y,'mean_accz':mn_acc_z,'std_accx':std_acc_x,
                            'std_accy':std_acc_y,'std_accz':std_acc_z,'integral_accx':itgl_acc_x,'integral_accy':itgl_acc_y,
                            'integral_accz':itgl_acc_z,'mean_emg':mn_emg,'std_emg':std_emg,'drange_emg':dr_emg,'integral_emg':itgl_emg,
                            'median_emg':med_emg,'percentile_10_emg':per_10_emg,'percentile_90_emg':per_90_emg,'number_peaks':peaks.size,
                            'mean_peak_amplitudes':mn_pk_amp,'std_peak_amplitude':std_pk_amp,'sum_peak_amplitude':sum_pk_amp,'mean_temp':mn_temp,
                            'std_temp':std_temp,'drange_temp':dr_temp,'max_temp':max_temp,'min_temp':min_temp,'mean_eda':mn_eda,'std_eda':std_eda,
                            'max_eda':max_eda,'min_eda':min_eda,'drange_eda':dr_eda,'mean_scr_peaks':mn_scr,'std_scr_peaks':std_scr,'total_scr_peaks':num_scr,
                            'sum_scr_peaks':sum_scr,'area_peaks_scr':ar_scr},ignore_index = True)

c:\users\dell\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:52: PeakPropertyWarning: some peaks have a width of 0


In [11]:
df_new

,area_peaks_scr,drange_eda,drange_emg,drange_temp,integral_accx,integral_accy,integral_accz,integral_emg,max_eda,max_temp,...,std_accy,std_accz,std_eda,std_emg,std_peak_amplitude,std_scr_peaks,std_temp,sum_peak_amplitude,sum_scr_peaks,total_scr_peaks
0,0.112629,-0.673155,1.060819,-1.306436,2538.461503,-363.758003,-776.691500,15870.722961,0.068069,4.026794,...,0.011992,0.015748,0.019296,0.102055,0.101754,0.011924,2.079894,5118.066025,8.892151,525.0
1,0.112629,-0.673155,1.060819,-1.306436,2538.453503,-363.765203,-776.683300,15870.381927,0.068069,4.026794,...,0.011989,0.015752,0.019297,0.102001,0.101588,0.011924,2.080084,5112.346649,8.892151,525.0
2,0.112629,-0.673155,1.060819,-1.306436,2538.444203,-363.772403,-776.673900,15870.039368,0.068069,4.026794,...,0.011987,0.015755,0.019296,0.101949,0.101628,0.011924,2.080279,5117.728043,8.892151,525.0
3,0.112549,-0.673155,1.060819,-1.306436,2538.433803,-363.778503,-776.667000,15869.692993,0.068069,4.026794,...,0.011985,0.015758,0.019298,0.101903,0.101628,0.011934,2.080476,5117.728043,8.879517,524.0
4,0.112686,-0.673155,1.060819,-1.306436,2538.421603,-363.784503,-776.662800,15869.347191,0.068069,4.026794,...,0.011982,0.015759,0.019297,0.101847,0.101443,0.011923,2.080664,5112.001038,8.896042,525.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,0.092454,-0.760211,1.046244,-0.812056,2513.062194,-380.834801,-859.100199,15250.800705,0.072418,4.008484,...,0.010694,0.013198,0.017371,0.062138,0.063593,0.011326,2.436361,5107.098007,7.908051,542.0
2884,0.092604,-0.760211,1.046244,-0.812056,2513.046994,-380.834001,-859.160099,15250.611305,0.072418,4.008484,...,0.010695,0.013175,0.017373,0.062132,0.063524,0.011316,2.436366,5101.699448,7.925354,543.0
2885,0.092604,-0.760211,1.046244,-0.812056,2513.032394,-380.835101,-859.215799,15250.423622,0.072418,4.008484,...,0.010694,0.013153,0.017372,0.062124,0.063546,0.011316,2.436357,5106.903839,7.925354,543.0
2886,0.092604,-0.760211,1.046244,-0.812056,2513.017894,-380.838201,-859.275199,15250.245857,0.072418,4.008484,...,0.010692,0.013131,0.017371,0.062133,0.063546,0.011316,2.436351,5106.903839,7.925354,543.0


In [12]:
targ = pd.DataFrame(columns = ['target'])

In [13]:
for i in range(4):
               y = (i+1)*np.ones(int(len(df_new)/4))
               targ = targ.append(pd.DataFrame(y))

In [14]:
targ = targ.iloc[:,0]

In [15]:
targ = targ.reset_index()

In [16]:
targ = targ.iloc[:,-1]

In [17]:
targ

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2883    4.0
2884    4.0
2885    4.0
2886    4.0
2887    4.0
Name: 0, Length: 2888, dtype: float64

In [18]:
df1 = pd.concat([df_new,targ],axis = 1)

In [19]:
df1.iloc[:,-1]

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2883    4.0
2884    4.0
2885    4.0
2886    4.0
2887    4.0
Name: 0, Length: 2888, dtype: float64

In [20]:
df1 = df1.rename(columns={0: "target"})

In [21]:
from scipy import stats
df1 = df1[(np.abs(stats.zscore(df1)) < 3).all(axis=1)]

In [22]:
df1

,area_peaks_scr,drange_eda,drange_emg,drange_temp,integral_accx,integral_accy,integral_accz,integral_emg,max_eda,max_temp,...,std_accz,std_eda,std_emg,std_peak_amplitude,std_scr_peaks,std_temp,sum_peak_amplitude,sum_scr_peaks,total_scr_peaks,target
0,0.112629,-0.673155,1.060819,-1.306436,2538.461503,-363.758003,-776.691500,15870.722961,0.068069,4.026794,...,0.015748,0.019296,0.102055,0.101754,0.011924,2.079894,5118.066025,8.892151,525.0,1.0
1,0.112629,-0.673155,1.060819,-1.306436,2538.453503,-363.765203,-776.683300,15870.381927,0.068069,4.026794,...,0.015752,0.019297,0.102001,0.101588,0.011924,2.080084,5112.346649,8.892151,525.0,1.0
2,0.112629,-0.673155,1.060819,-1.306436,2538.444203,-363.772403,-776.673900,15870.039368,0.068069,4.026794,...,0.015755,0.019296,0.101949,0.101628,0.011924,2.080279,5117.728043,8.892151,525.0,1.0
3,0.112549,-0.673155,1.060819,-1.306436,2538.433803,-363.778503,-776.667000,15869.692993,0.068069,4.026794,...,0.015758,0.019298,0.101903,0.101628,0.011934,2.080476,5117.728043,8.879517,524.0,1.0
4,0.112686,-0.673155,1.060819,-1.306436,2538.421603,-363.784503,-776.662800,15869.347191,0.068069,4.026794,...,0.015759,0.019297,0.101847,0.101443,0.011923,2.080664,5112.001038,8.896042,525.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,0.092454,-0.760211,1.046244,-0.812056,2513.062194,-380.834801,-859.100199,15250.800705,0.072418,4.008484,...,0.013198,0.017371,0.062138,0.063593,0.011326,2.436361,5107.098007,7.908051,542.0,4.0
2884,0.092604,-0.760211,1.046244,-0.812056,2513.046994,-380.834001,-859.160099,15250.611305,0.072418,4.008484,...,0.013175,0.017373,0.062132,0.063524,0.011316,2.436366,5101.699448,7.925354,543.0,4.0
2885,0.092604,-0.760211,1.046244,-0.812056,2513.032394,-380.835101,-859.215799,15250.423622,0.072418,4.008484,...,0.013153,0.017372,0.062124,0.063546,0.011316,2.436357,5106.903839,7.925354,543.0,4.0
2886,0.092604,-0.760211,1.046244,-0.812056,2513.017894,-380.838201,-859.275199,15250.245857,0.072418,4.008484,...,0.013131,0.017371,0.062133,0.063546,0.011316,2.436351,5106.903839,7.925354,543.0,4.0


In [25]:
df1 = df1.drop_duplicates(keep = 'first')

In [26]:
df1

,area_peaks_scr,drange_eda,drange_emg,drange_temp,integral_accx,integral_accy,integral_accz,integral_emg,max_eda,max_temp,...,std_accz,std_eda,std_emg,std_peak_amplitude,std_scr_peaks,std_temp,sum_peak_amplitude,sum_scr_peaks,total_scr_peaks,target
0,0.112629,-0.673155,1.060819,-1.306436,2538.461503,-363.758003,-776.691500,15870.722961,0.068069,4.026794,...,0.015748,0.019296,0.102055,0.101754,0.011924,2.079894,5118.066025,8.892151,525.0,1.0
1,0.112629,-0.673155,1.060819,-1.306436,2538.453503,-363.765203,-776.683300,15870.381927,0.068069,4.026794,...,0.015752,0.019297,0.102001,0.101588,0.011924,2.080084,5112.346649,8.892151,525.0,1.0
2,0.112629,-0.673155,1.060819,-1.306436,2538.444203,-363.772403,-776.673900,15870.039368,0.068069,4.026794,...,0.015755,0.019296,0.101949,0.101628,0.011924,2.080279,5117.728043,8.892151,525.0,1.0
3,0.112549,-0.673155,1.060819,-1.306436,2538.433803,-363.778503,-776.667000,15869.692993,0.068069,4.026794,...,0.015758,0.019298,0.101903,0.101628,0.011934,2.080476,5117.728043,8.879517,524.0,1.0
4,0.112686,-0.673155,1.060819,-1.306436,2538.421603,-363.784503,-776.662800,15869.347191,0.068069,4.026794,...,0.015759,0.019297,0.101847,0.101443,0.011923,2.080664,5112.001038,8.896042,525.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,0.092454,-0.760211,1.046244,-0.812056,2513.062194,-380.834801,-859.100199,15250.800705,0.072418,4.008484,...,0.013198,0.017371,0.062138,0.063593,0.011326,2.436361,5107.098007,7.908051,542.0,4.0
2884,0.092604,-0.760211,1.046244,-0.812056,2513.046994,-380.834001,-859.160099,15250.611305,0.072418,4.008484,...,0.013175,0.017373,0.062132,0.063524,0.011316,2.436366,5101.699448,7.925354,543.0,4.0
2885,0.092604,-0.760211,1.046244,-0.812056,2513.032394,-380.835101,-859.215799,15250.423622,0.072418,4.008484,...,0.013153,0.017372,0.062124,0.063546,0.011316,2.436357,5106.903839,7.925354,543.0,4.0
2886,0.092604,-0.760211,1.046244,-0.812056,2513.017894,-380.838201,-859.275199,15250.245857,0.072418,4.008484,...,0.013131,0.017371,0.062133,0.063546,0.011316,2.436351,5106.903839,7.925354,543.0,4.0


In [28]:
df1.iloc[:,-1]

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2883    4.0
2884    4.0
2885    4.0
2886    4.0
2887    4.0
Name: target, Length: 2873, dtype: float64

In [29]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(df1.iloc[:,:-1], df1.iloc[:,-1])

C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.base module is

In [30]:
X_resampled 

array([[ 1.12628949e-01, -6.73155274e-01,  1.06081863e+00, ...,
         5.11806602e+03,  8.89215088e+00,  5.25000000e+02],
       [ 1.12628949e-01, -6.73155274e-01,  1.06081863e+00, ...,
         5.11234665e+03,  8.89215088e+00,  5.25000000e+02],
       [ 1.12628949e-01, -6.73155274e-01,  1.06081863e+00, ...,
         5.11772804e+03,  8.89215088e+00,  5.25000000e+02],
       ...,
       [ 9.50717619e-02, -7.60211437e-01,  1.04929584e+00, ...,
         4.96245461e+03,  7.92819214e+00,  5.31000000e+02],
       [ 9.26039033e-02, -7.60211437e-01,  1.04624411e+00, ...,
         5.10690384e+03,  7.92535400e+00,  5.43000000e+02],
       [ 9.32667127e-02, -7.60211437e-01,  1.04929584e+00, ...,
         5.06253319e+03,  7.90686035e+00,  5.42000000e+02]])

In [31]:
y_resampled 

array([1., 1., 1., ..., 4., 4., 4.])

In [32]:
df1 = pd.concat([pd.DataFrame(X_resampled ),pd.DataFrame(y_resampled)],axis = 1)

In [33]:
df1

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,0
0,0.112629,-0.673155,1.060819,-1.306436,2538.461503,-363.758003,-776.691500,15870.722961,0.068069,4.026794,...,0.015748,0.019296,0.102055,0.101754,0.011924,2.079894,5118.066025,8.892151,525.0,1.0
1,0.112629,-0.673155,1.060819,-1.306436,2538.453503,-363.765203,-776.683300,15870.381927,0.068069,4.026794,...,0.015752,0.019297,0.102001,0.101588,0.011924,2.080084,5112.346649,8.892151,525.0,1.0
2,0.112629,-0.673155,1.060819,-1.306436,2538.444203,-363.772403,-776.673900,15870.039368,0.068069,4.026794,...,0.015755,0.019296,0.101949,0.101628,0.011924,2.080279,5117.728043,8.892151,525.0,1.0
3,0.112549,-0.673155,1.060819,-1.306436,2538.433803,-363.778503,-776.667000,15869.692993,0.068069,4.026794,...,0.015758,0.019298,0.101903,0.101628,0.011934,2.080476,5117.728043,8.879517,524.0,1.0
4,0.112686,-0.673155,1.060819,-1.306436,2538.421603,-363.784503,-776.662800,15869.347191,0.068069,4.026794,...,0.015759,0.019297,0.101847,0.101443,0.011923,2.080664,5112.001038,8.896042,525.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,0.091577,-0.760211,1.049296,-0.812056,2514.028695,-379.850501,-856.086999,15268.602943,0.072418,4.008484,...,0.014131,0.017316,0.061322,0.063304,0.011313,2.436458,5103.153229,7.867126,547.0,4.0
2884,0.095299,-0.760211,1.049296,-0.812056,2515.201794,-378.083801,-852.130399,15304.637718,0.072418,4.008484,...,0.015168,0.017671,0.058990,0.061136,0.011516,2.435884,4999.649429,8.003860,536.0,4.0
2885,0.095072,-0.760211,1.049296,-0.812056,2515.569594,-377.622401,-850.172699,15319.522858,0.072418,4.008484,...,0.015437,0.017571,0.057669,0.059844,0.011626,2.434536,4962.454605,7.928192,531.0,4.0
2886,0.092604,-0.760211,1.046244,-0.812056,2513.017894,-380.838201,-859.275199,15250.245857,0.072418,4.008484,...,0.013131,0.017371,0.062133,0.063546,0.011316,2.436351,5106.903839,7.925354,543.0,4.0


In [34]:
targ = df1.iloc[:,-1]
df1 = df1.iloc[:,:-1]

In [35]:
targ

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2883    4.0
2884    4.0
2885    4.0
2886    4.0
2887    4.0
Name: 0, Length: 2888, dtype: float64

In [36]:
df1

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,0.112629,-0.673155,1.060819,-1.306436,2538.461503,-363.758003,-776.691500,15870.722961,0.068069,4.026794,...,0.011992,0.015748,0.019296,0.102055,0.101754,0.011924,2.079894,5118.066025,8.892151,525.0
1,0.112629,-0.673155,1.060819,-1.306436,2538.453503,-363.765203,-776.683300,15870.381927,0.068069,4.026794,...,0.011989,0.015752,0.019297,0.102001,0.101588,0.011924,2.080084,5112.346649,8.892151,525.0
2,0.112629,-0.673155,1.060819,-1.306436,2538.444203,-363.772403,-776.673900,15870.039368,0.068069,4.026794,...,0.011987,0.015755,0.019296,0.101949,0.101628,0.011924,2.080279,5117.728043,8.892151,525.0
3,0.112549,-0.673155,1.060819,-1.306436,2538.433803,-363.778503,-776.667000,15869.692993,0.068069,4.026794,...,0.011985,0.015758,0.019298,0.101903,0.101628,0.011934,2.080476,5117.728043,8.879517,524.0
4,0.112686,-0.673155,1.060819,-1.306436,2538.421603,-363.784503,-776.662800,15869.347191,0.068069,4.026794,...,0.011982,0.015759,0.019297,0.101847,0.101443,0.011923,2.080664,5112.001038,8.896042,525.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,0.091577,-0.760211,1.049296,-0.812056,2514.028695,-379.850501,-856.086999,15268.602943,0.072418,4.008484,...,0.011079,0.014131,0.017316,0.061322,0.063304,0.011313,2.436458,5103.153229,7.867126,547.0
2884,0.095299,-0.760211,1.049296,-0.812056,2515.201794,-378.083801,-852.130399,15304.637718,0.072418,4.008484,...,0.011119,0.015168,0.017671,0.058990,0.061136,0.011516,2.435884,4999.649429,8.003860,536.0
2885,0.095072,-0.760211,1.049296,-0.812056,2515.569594,-377.622401,-850.172699,15319.522858,0.072418,4.008484,...,0.010971,0.015437,0.017571,0.057669,0.059844,0.011626,2.434536,4962.454605,7.928192,531.0
2886,0.092604,-0.760211,1.046244,-0.812056,2513.017894,-380.838201,-859.275199,15250.245857,0.072418,4.008484,...,0.010692,0.013131,0.017371,0.062133,0.063546,0.011316,2.436351,5106.903839,7.925354,543.0


In [38]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(df1)
df1 = scaler.transform(df1)

In [40]:
df1 = pd.DataFrame(df1)

In [41]:
df1

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,2.676209,1.391982,1.315094,-0.103196,1.705528,1.736885,1.713112,2.008317,-1.699101,0.487772,...,1.747600,1.430354,2.841299,2.652181,2.598226,1.933798,0.083290,1.611198,2.655104,-0.047260
1,2.676209,1.391982,1.315094,-0.103196,1.704492,1.735254,1.713438,2.006328,-1.699101,0.487772,...,1.745941,1.433443,2.841812,2.648556,2.587104,1.933798,0.083792,1.578038,2.655104,-0.047260
2,2.676209,1.391982,1.315094,-0.103196,1.703288,1.733624,1.713812,2.004330,-1.699101,0.487772,...,1.744067,1.435960,2.840719,2.645141,2.589758,1.933798,0.084309,1.609239,2.655104,-0.047260
3,2.663483,1.391982,1.315094,-0.103196,1.701941,1.732242,1.714086,2.002309,-1.699101,0.487772,...,1.742679,1.437607,2.844367,2.642114,2.589758,1.977238,0.084828,1.609239,2.617004,-0.166777
4,2.685255,1.391982,1.315094,-0.103196,1.700361,1.730883,1.714253,2.000292,-1.699101,0.487772,...,1.740665,1.438417,2.842493,2.638394,2.577430,1.927350,0.085327,1.576034,2.666838,-0.047260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2883,-0.680698,-1.600239,-0.371022,0.893542,-1.458734,-1.907664,-1.444066,-1.503715,0.588546,-2.066120,...,1.125656,0.148246,-1.111450,-0.044576,0.030709,-0.751431,1.026157,1.524736,-0.435946,2.582110
2884,-0.087122,-1.600239,-0.371022,0.893542,-1.306807,-1.507550,-1.286731,-1.293532,0.588546,-2.066120,...,1.153053,0.970192,-0.402952,-0.198941,-0.114042,0.139607,1.024640,0.924633,-0.023613,1.267425
2885,-0.123375,-1.600239,-0.371022,0.893542,-1.259174,-1.403055,-1.208883,-1.206711,0.588546,-2.066120,...,1.052191,1.183669,-0.604071,-0.286430,-0.200310,0.623358,1.021076,0.708982,-0.251797,0.669840
2886,-0.516888,-1.600239,-0.817578,0.893542,-1.589642,-2.131353,-1.570846,-1.610788,0.588546,-2.066120,...,0.862083,-0.643881,-1.003018,0.009087,0.046886,-0.737284,1.025876,1.546481,-0.260356,2.104042


In [42]:
from sklearn.model_selection import train_test_split
df_train, df_test, targ_train, targ_test = train_test_split(df1, targ, test_size=0.2, random_state=42)

In [66]:
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from itertools import product
from sklearn.neighbors import KNeighborsClassifier

In [60]:
targ_train = pd.DataFrame(targ_train)
targ_test = pd.DataFrame(targ_test)

In [61]:
kf = KFold(n_splits = 5)
i = 0
for train,test in kf.split(df_train):
    lda =  LinearDiscriminantAnalysis()
    data_t,data_v,targ_t,targ_v = df_train.iloc[train,:],df_train.iloc[test,:],targ_train.iloc[train,:],targ_train.iloc[test,:]
    lda.fit(data_t,targ_t)
    i+=1
    print(i)

C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1
2
3
4
5


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d

In [62]:
lda.score(df_test,targ_test)

0.9878892733564014

In [63]:
kf = KFold(n_splits = 5)
i = 0
for train,test in kf.split(df_train):
    qda =  QuadraticDiscriminantAnalysis()
    data_t,data_v,targ_t,targ_v = df_train.iloc[train,:],df_train.iloc[test,:],targ_train.iloc[train,:],targ_train.iloc[test,:]
    qda.fit(data_t,targ_t)
    i+=1
    print(i)

C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1
2
3
4
5


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\discriminant

In [64]:
qda.score(df_test,targ_test)

0.9757785467128027

In [67]:
kf = KFold(n_splits = 5)
i = 0
for train,test in kf.split(df_train):
    svc = SVC(gamma = 'auto')
    data_t,data_v,targ_t,targ_v = df_train.iloc[train,:],df_train.iloc[test,:],targ_train.iloc[train,:],targ_train.iloc[test,:]
    svc.fit(data_t,targ_t)
    i+=1
    print(i)

C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1
2
3
4
5


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [68]:
svc.score(df_test,targ_test)

0.9878892733564014

In [69]:
from sklearn.neural_network import MLPClassifier

In [79]:
kf = KFold(n_splits = 20)
i = 0
for train,test in kf.split(df_train):
    nn = MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)
    data_t,data_v,targ_t,targ_v = df_train.iloc[train,:],df_train.iloc[test,:],targ_train.iloc[train,:],targ_train.iloc[test,:]
    nn.fit(data_t,targ_t)
    i+=1
    print(i)

C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


...

C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


17
18
19


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


20


C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\sklearn\neural_network\_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [80]:
nn.score(df_test,targ_test)

0.9411764705882353